# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


In [1]:
import keras
from keras.datasets import cifar10
from keras.utils import np_utils
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Activation, Dropout, Flatten, Reshape, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train.shape

(50000, 32, 32, 3)

In [4]:
x_train_lt5 = x_train[y_train[y_train < 5]]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test[y_test < 5]]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train[y_train >= 5]]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test[y_test >= 5]]
y_test_gte5 = y_test[y_test >= 5] - 5

In [5]:
print(x_train_lt5.shape)
print(x_test_lt5.shape)

(25000, 32, 32, 3)
(5000, 32, 32, 3)


In [10]:
x_train_lt5 = x_train_lt5.astype('float32')
x_test_lt5 = x_test_lt5.astype('float32')
x_train_lt5 /= x_train_lt5/255
x_test_lt5 /= x_test_lt5/255

x_train_gte5 = x_train_gte5.astype('float32')
x_test_gte5 = x_test_gte5.astype('float32')
x_train_gte5 /= x_train_gte5/255
x_test_gte5 /= x_test_gte5/255

### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [7]:
y_train_lt5 = np_utils.to_categorical(y_train_lt5, 10)
y_test_lt5 = np_utils.to_categorical(y_test_lt5, 10)

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [8]:
conv_layers = [
    Conv2D(32, (3,3),input_shape=(32, 32, 3)),
    Activation('relu'),
    BatchNormalization(),
    Conv2D(32, (3, 3)),
    Activation('relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),
    Conv2D(64, (3, 3)),
    Activation('relu'),
    BatchNormalization(),
    Conv2D(64, (3, 3)),
    Activation('relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.4)
]
dense_layers = [
    Flatten(),
    Dense(128),
    Activation('relu'),
    BatchNormalization(),
    Dense(10),
    Activation('softmax')
]

model = Sequential(conv_layers + dense_layers)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

early_stopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, verbose=1, mode='auto')
callback_list = [early_stopping]

In [9]:
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True
)

datagen.fit(x_train_lt5)

In [13]:
BATCH_SIZE = 100
EPOCHS = 5

model.fit_generator(
    datagen.flow(x_train_lt5, y_train_lt5, batch_size=100),
    epochs=5,
    verbose=1,
    shuffle=True,
    validation_data=(x_test_lt5, y_test_lt5),
    callbacks=callback_list
)

Epoch 1/5
1250/1250 [==============================] - 2003s 2s/step - loss: 0.9297 - acc: 0.6650 - val_loss: 0.7293 - val_acc: 0.7422
Epoch 2/5
1250/1250 [==============================] - 1772s 1s/step - loss: 0.6429 - acc: 0.7746 - val_loss: 0.5594 - val_acc: 0.8114
Epoch 3/5
1250/1250 [==============================] - 1760s 1s/step - loss: 0.5632 - acc: 0.8092 - val_loss: 0.5163 - val_acc: 0.8320
Epoch 4/5
1250/1250 [==============================] - 1729s 1s/step - loss: 0.5237 - acc: 0.8306 - val_loss: 0.5360 - val_acc: 0.8326
Epoch 5/5
1250/1250 [==============================] - 1707s 1s/step - loss: 0.4962 - acc: 0.8449 - val_loss: 0.5164 - val_acc: 0.8462


In [18]:
model.evaluate(x_test_lt5, y_test_lt5)

5000/5000 [==============================] - 15s 3ms/step


[0.5163634763717652, 0.8462]

### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [11]:
for layer in conv_layers:
    layer.trainable = False
    
for layer in dense_layers:
    layer.trainable = True
    
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [22]:
model.fit_generator(
    datagen.flow(x_train_gte5, y_train_gte5, batch_size=100),
    epochs=5,
    verbose=1,
    shuffle=True,
    validation_data=(x_test_gte5, y_test_gte5),
    callbacks=callback_list
)

Epoch 1/5
625/625 [==============================] - 425s 679ms/step - loss: 0.4509 - acc: 0.8649 - val_loss: 0.5239 - val_acc: 0.8450
Epoch 2/5
625/625 [==============================] - 448s 716ms/step - loss: 0.4455 - acc: 0.8660 - val_loss: 0.5158 - val_acc: 0.8466
Epoch 3/5
625/625 [==============================] - 450s 720ms/step - loss: 0.4440 - acc: 0.8672 - val_loss: 0.5066 - val_acc: 0.8516
Epoch 4/5
625/625 [==============================] - 525s 840ms/step - loss: 0.4382 - acc: 0.8691 - val_loss: 0.5218 - val_acc: 0.8478
Epoch 5/5
625/625 [==============================] - 649s 1s/step - loss: 0.4371 - acc: 0.8699 - val_loss: 0.5295 - val_acc: 0.8456


In [23]:
model.evaluate(x_test_9, y_test_9)

5000/5000 [==============================] - 20s 4ms/step


[0.5294680576324463, 0.8456]

# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [24]:
from sklearn.datasets import fetch_20newsgroups

In [25]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [26]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### 8. Test data

In [27]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [28]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2], dtype=int64)

In [29]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [30]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer()
tf_train = tf.fit_transform(twenty_train.data)
tf_test = tf.transform(twenty_test.data)

In [34]:
tf.get_feature_names()

['00',
 '000',
 '0000',
 '0000001200',
 '000005102000',
 '0001',
 '000100255pixel',
 '00014',
 '000406',
 '0007',
 '000usd',
 '0010',
 '001004',
 '0010580b',
 '001125',
 '001200201pixel',
 '0014',
 '001642',
 '00196',
 '002',
 '0028',
 '003258u19250',
 '0033',
 '0038',
 '0039',
 '004021809',
 '004158',
 '004627',
 '0049',
 '00500',
 '005148',
 '00630',
 '008561',
 '0094',
 '00am',
 '00index',
 '00pm',
 '01',
 '0100',
 '010116',
 '010702',
 '011255',
 '011308pxf3',
 '011605',
 '011720',
 '012019',
 '012536',
 '012946',
 '013',
 '013034',
 '0131',
 '013423tan102',
 '013657',
 '0138',
 '013846',
 '0150',
 '015518',
 '01580',
 '015931',
 '01720',
 '01752',
 '0179',
 '01821',
 '01830',
 '0184',
 '01852',
 '01854',
 '01890',
 '0195',
 '0199',
 '01a',
 '01wb',
 '02',
 '020',
 '0200',
 '020504',
 '020637',
 '020646',
 '020701tan102',
 '020751',
 '02115',
 '02118',
 '02139',
 '02142',
 '02154',
 '02160',
 '021635',
 '021708',
 '02173',
 '0223',
 '02254',
 '022621tan102',
 '023044',
 '0235',
 '0

### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [37]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

logreg = LogisticRegression()
logreg.fit(tf_train, twenty_train.target)
y_pred_class = logreg.predict(tf_test)

In [38]:
print('Train Accuracy:', metrics.accuracy_score(twenty_train.target, logreg.predict(tf_train)))
print('Test Accuracy:', metrics.accuracy_score(twenty_test.target, y_pred_class))

Train Accuracy: 0.9827204253433761
Test Accuracy: 0.8868175765645806
